<h1>
  <b>
    Creación y entrenamiento del modelo de detección de comida saludable
  </b>
</h1>

---

<h3>
  Montar el modelo en Google Drive
</h3>

In [ ]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


---
<h3>
  Cargar las imágenes etiquetadas desde Google Drive
</h3>

In [ ]:
import os
import cv2
import numpy as np

# Rutas a las carpetas de imágenes
ruta_saludable = '/content/drive/My Drive/Colab/data/saludable'
ruta_no_saludable = '/content/drive/My Drive/Colab/data/no_saludable'

# Función para cargar y preprocesar imágenes desde carpetas separadas
def cargar_imagenes_desde_carpetas(ruta_saludable, ruta_no_saludable):
    datos = []
    for ruta, etiqueta in [(ruta_saludable, 0), (ruta_no_saludable, 1)]:
        for archivo in os.listdir(ruta):
            # Leer la imagen y preprocesarla (ajustar según tus necesidades)
            imagen = cv2.imread(os.path.join(ruta, archivo))
            imagen = cv2.resize(imagen, (224, 224))
            imagen = imagen / 255.0  # Normalizar
            datos.append((imagen, etiqueta))
    return datos

# Cargar imágenes desde carpetas
datos = cargar_imagenes_desde_carpetas(ruta_saludable, ruta_no_saludable)

# Dividir los datos en características (X) y etiquetas (y)
X, y = zip(*datos)

# Convertir a arreglos NumPy
X = np.array(X)
y = np.array(y)

# Dividir los datos en entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

---
<h3>
  Entrenamiento del modelo
</h3>

In [ ]:
# Entrenar el modelo
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Cargar el modelo base pre-entrenado (VGG16 en este caso)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base para que no se actualicen durante el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

# Agregar capas personalizadas para la clasificación
x = base_model.output
x = Flatten()(x)
predictions = Dense(2, activation='softmax')(x)  # 2 clases: saludable o no saludable

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
12/12 [==============================] - 18s 552ms/step - loss: 1.5061 - accuracy: 0.5419 - val_loss: 0.5955 - val_accuracy: 0.7500
Epoch 2/10
12/12 [==============================] - 2s 155ms/step - loss: 0.5755 - accuracy: 0.7461 - val_loss: 0.3400 - val_accuracy: 0.8333
Epoch 3/10
12/12 [==============================] - 2s 155ms/step - loss: 0.2503 - accuracy: 0.8927 - val_loss: 0.2553 - val_accuracy: 0.9062
Epoch 4/10
12/12 [==============================] - 2s 179ms/step - loss: 0.1335 - accuracy: 0.9529 - val_loss: 0.3379 - val_accuracy: 0.8021
Epoch 5/10
12/12 [==============================] - 2s 154ms/step - loss: 0.0743 - accuracy: 0.9895 - val_loss: 0.2446 - val_accuracy: 0.9271
Epoch 6/10
12/12 [==============================] - 2s 158ms/step - loss: 0.0432 - accuracy: 0.9974 - val_loss: 0.2007 - val_accuracy: 0.9375
Epoch 7/10
12/12 [==============================] - 2s 183ms/step - loss: 0.0304 -

---
<h3>  
    Comporbar la precisión del modelo
</h3>

In [ ]:
# Evaluar el modelo con el conjunto de validación
score = model.evaluate(X_val, y_val)
print("Precisión en el conjunto de validación: {:.2f}%".format(score[1] * 100))

3/3 [==============================] - 0s 122ms/step - loss: 0.1868 - accuracy: 0.9271
Precisión en el conjunto de validación: 92.71%


---
<h3>
    Elegir imagen para probar el modelo y mostsrar su resultado
</h3>

In [ ]:
nueva_imagen = cv2.imread('/content/drive/My Drive/Colab/data/pruebas/prueba2.jpg')
nueva_imagen = cv2.resize(nueva_imagen, (224, 224))
nueva_imagen = nueva_imagen / 255.0  # Normalizar
nueva_imagen = np.expand_dims(nueva_imagen, axis=0)

# Hacer la predicción
prediccion = model.predict(nueva_imagen)

# La salida de la red es un vector de probabilidades para cada clase
# Si tienes dos clases (saludable y no saludable), puedes usar argmax para obtener la clase predicha
clase_predicha = np.argmax(prediccion, axis=1)

# Imprimir el resultado
if clase_predicha == 0:
    print("La comida en la imagen es saludable.")
else:
    print("La comida en la imagen no es saludable.")

1/1 [==============================] - 0s 18ms/step
La comida en la imagen es saludable.


---
<h3>  
    Guardar el modelo como ".h5" para usar como API
</h3>

In [ ]:
model.save('modelo_entrenado.h5')
!ls

drive  modelo_entrenado.h5  sample_data


---